# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

21:42:28 | NewsletterAgent.newsletter_agent | INFO | Test info message
21:42:28 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
21:42:28 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))
try:
    # set up state
    session_id = 'test_newsletter_20250928201328725764'
    step_name = 'step_05_cluster_by_topic'
#     del session_id
except Exception as e:
    print(e)

do_download=False
process_since=None
process_since='2025-09-28 11:00:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=True
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=True
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

21:42:28 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Using provided state with 168 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20250928201328725764
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250928201328725764


In [6]:
agent.state.get_status()


{'headlines': {'total': 168},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_06_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 55.55555555555556,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [16]:
state.get_current_step()


'step_01_fetch_urls'

In [17]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


20:13:34 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
20:13:34 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status


⏱️  Total execution time: 5.68s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- Status summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step details:
- Step 1: Fetch Urls: not_started
- Step 2: Filter Urls: not_started
- Step 3: Download Articles: not_started
- Step 4: Extract Summaries: not_started
- Step 5: Cluster By Topic: not_started
- Step 6: Rate Articles: not_started
- Step 7: Select Sections: not_started
- Step 8: Draft Sections: not_started
- Step 9: Finalize Newsletter: not_started

Would you like me to start Step 1 (gather URLs) and continue through the workflow, or run a specific step?


In [18]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


20:13:42 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
20:13:42 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status
20:13:43 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting Step 1: Gather URLs
2025-09-28 20:13:43,903 - fetcher_5084558864 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-28 20:13:43,919 - fetcher_5084558864 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-28 20:13:43,919 - fetcher_5084558864 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-28 20:13:43,920 - fetcher_5084558864 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-28 20:13:43,920 - fetcher_5084558864 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-28 20:13:43,920 - fetcher_5084558864 - DEBU

▶ Starting Step 1: step_01_fetch_urls


2025-09-28 20:13:44,185 - fetcher_5084558864 - INFO - [fetch_html] Parsed HTML file: download/sources/Feedly_AI.html
2025-09-28 20:13:44,185 - fetcher_5084558864 - INFO - [fetch_html] HTML fetch successful for Feedly AI: 105 articles
2025-09-28 20:13:44,186 - fetcher_5084558864 - INFO - [fetch_rss] Fetching RSS from Hacker News: https://news.ycombinator.com/rss
2025-09-28 20:13:44,186 - fetcher_5084558864 - INFO - [fetch_rss] Fetching RSS from HackerNoon: https://hackernoon.com/tagged/ai/feed
2025-09-28 20:13:44,188 - fetcher_5084558864 - INFO - [fetch_rss] Fetching RSS from New York Times: https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml
2025-09-28 20:13:44,189 - fetcher_5084558864 - INFO - [fetch_html] Using existing HTML file from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+

,source,url
0,Ars Technica,20
1,Bloomberg,27
2,Business Insider,15
3,FT,50
4,Feedly AI,102
5,Hacker News,30
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,28
9,Reddit,49


,source,title,url,published,rss_summary,id
0,Ars Technica,Why LA Comic Con thought making an AI-powered ...,https://arstechnica.com/ai/2025/09/why-la-comi...,"Sat, 27 Sep 2025 11:00:07 +0000",“I suppose if we do it and thousands of fans… ...,0
1,Ars Technica,Can AI detect hedgehogs from space? Maybe if y...,https://arstechnica.com/ai/2025/09/can-ai-dete...,"Fri, 26 Sep 2025 22:22:13 +0000",Cambridge researchers use satellite-based bram...,1
2,Ars Technica,YouTube Music is testing AI hosts that will in...,https://arstechnica.com/google/2025/09/youtube...,"Fri, 26 Sep 2025 21:05:30 +0000",YouTube Labs will be a place to preview all th...,2
3,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,3
4,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,4
...,...,...,...,...,...,...
556,NewsAPI,Libra split? More clubs condemn Flamengo’s mov...,https://onefootball.com/en/news/libra-split-mo...,2025-09-27T22:27:00Z,NaN,556
557,NewsAPI,Why Advertisers Are Returning to Big Oil Despi...,https://oilprice.com/Energy/Energy-General/Why...,2025-09-27T23:00:00Z,NaN,557
558,NewsAPI,Show HN: AI Video Generator app for iOS (looki...,https://apps.apple.com/us/app/ai-video-generat...,2025-09-27T23:05:00Z,NaN,558
559,NewsAPI,Overcoming all odds for Oxford,https://www.thestar.com.my/news/education/2025...,2025-09-27T23:00:00Z,NaN,559


20:13:44 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed Step 1: Gathered 646 articles
20:13:46 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
20:13:46 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status


⏱️  Total execution time: 9.46s
📊 Final result:
Step 1 completed.

Summary:
- Fetched URLs and headlines from sources (RSS): 646 collected, 561 stored in persistent state.
- Workflow progress: 11.1% (1/9 complete)
- Next step: Step 2 — Filter URLs (to AI-related content)

Would you like me to continue and run Step 2 (filter URLs) now?


In [19]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Why LA Comic Con thought making an AI-powered ...,https://arstechnica.com/ai/2025/09/why-la-comi...,"Sat, 27 Sep 2025 11:00:07 +0000",“I suppose if we do it and thousands of fans… ...,0
1,Ars Technica,Can AI detect hedgehogs from space? Maybe if y...,https://arstechnica.com/ai/2025/09/can-ai-dete...,"Fri, 26 Sep 2025 22:22:13 +0000",Cambridge researchers use satellite-based bram...,1
2,Ars Technica,YouTube Music is testing AI hosts that will in...,https://arstechnica.com/google/2025/09/youtube...,"Fri, 26 Sep 2025 21:05:30 +0000",YouTube Labs will be a place to preview all th...,2
3,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,3
4,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,4
...,...,...,...,...,...,...
556,NewsAPI,Libra split? More clubs condemn Flamengo’s mov...,https://onefootball.com/en/news/libra-split-mo...,2025-09-27T22:27:00Z,NaN,556
557,NewsAPI,Why Advertisers Are Returning to Big Oil Despi...,https://oilprice.com/Energy/Energy-General/Why...,2025-09-27T23:00:00Z,NaN,557
558,NewsAPI,Show HN: AI Video Generator app for iOS (looki...,https://apps.apple.com/us/app/ai-video-generat...,2025-09-27T23:05:00Z,NaN,558
559,NewsAPI,Overcoming all odds for Oxford,https://www.thestar.com.my/news/education/2025...,2025-09-27T23:00:00Z,NaN,559


In [20]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
4,Feedly AI,102
12,The Register,50
3,FT,50
6,HackerNoon,50
9,Reddit,49
5,Hacker News,30
8,NewsAPI,28
16,Washington Post,28
1,Bloomberg,27
7,New York Times,23


In [21]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting Step 2: Filter URLs
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔍 Filtering 561 headlines...
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔄 Checking for duplicates seen before 2025-09-28T11:00:00
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔍 Filtering 561 articles for dupes.
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | URL deduplication with process_since: 234 URLs filtered (seen before 2025-09-28T11:00:00), 327 new URLs remain
20:13:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 🔍 Filtering 561 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/ai/2025/09/why-la-comic-con-thought-making-an-ai-powered-stan-lee-hologram-was-a-good-idea/
found before cutoff
checking https://arstechnica.com/ai/2025/09/can-ai-detect-hedgehogs-from-space-maybe-if-you-find-brambles-first/
found before cutoff
checking https://arstechnica.com/google/2025/09/youtube-music-is-testing-ai-hosts-that-will-interrupt-your-tunes/
found before cutoff
checking https://arstechnica.com/ai/2025/09/chatgpt-pulse-delivers-morning-updates-based-on-your-chat-history/
found before cutoff
checking https://arstechnica.com/information-technology/2025/09/experts-urge-caution-about-using-chatgpt-to-pick-stocks/
found before cutoff
checking https://arstechnica.com/google/2025/09/google-deepmind-unveils-its-first-thinking-robotics-ai/
found before cutoff
checking https://arstechnica.com/science/2025/09/deepminds-robotic-ballet-an-ai-for-coordinating-manufacturing-robots/
found before cutoff
checking https

20:14:04 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed Step 2: 172 AI-related articles, 234 duplicates removed


⏱️  Total execution time: 16.86s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 234 duplicate URLs, classified 327 new articles, found 172 AI-related.


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

20:14:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting Step 3: Download Articles
20:14:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting concurrent scraping of 172 AI-related articles


▶ Starting Step 3: step_03_download_articles


20:14:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Launching browser for 172 URLs with 16 concurrent workers
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 0 fetching 1 of 172 https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html)
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html to download/html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Downloading https://www.yahoo.com/news/articles/uae-president-meets-openai-ceo-220423052.html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 1 fetching 2 of 172 https://www.liquid.ai/press/liquid-unveils-nanos-extremely-small-foundat

20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 6 completed https://www.wsj.com/tech/ai/ai-bubble-building-spree-55ee6128 with status: success
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 6 fetching 10 of 172 https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy)
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy to download/html
20:14:21 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Downloading https://www.theverge.com/news/787042/trump-posts-then-pulls-bizarre-ai-video-promoting-medbed-conspiracy
20:14:21 | NewsletterAgent.test_new

20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(https://natlawreview.com/article/all-china-patent-attorneys-association-bans-members-using-ai-generate-patent)
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping https://natlawreview.com/article/all-china-patent-attorneys-association-bans-members-using-ai-generate-patent to download/html
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Downloading https://natlawreview.com/article/all-china-patent-attorneys-association-bans-members-using-ai-generate-patent
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 11 fetching 20 of 172 http://www.techmeme.com/250928/p14#a250928p14
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scrape_url(http://www.techmeme.com/250928/p14#a250928p14)
20:14:23 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | scraping http://www.techmeme.com/250928/p14#a2509

...

20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | ERROR | Unexpected error scraping https://www.ft.com/content/c610a0a0-99b2-47eb-9536-e948c80a550e: Mouse.move: Target page, context or browser has been closed
Browser logs:

[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript error: , line 0: TypeError: NetworkError when attempting to fetch resource.
[pid=66632][err] JavaScript erro

20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 12 completed http://www.techmeme.com/250928/p13#a250928p13 with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 6 completed https://www.thewrap.com/donald-trump-posts-deletes-ai-generated-fox-news-medbeds/ with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 3 completed https://www.rcrwireless.com/20250928/chipsets/extension-voice-ai-era with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 5 completed https://news.google.com/rss/articles/CBMimgFBVV95cUxNMHF2eHRDUGhBOFpmSi1URFNGZExMZjNoNGJSTXNDX2ZMLW5uN29nc3Q0RVZkQndWaS11VjJXeFcyeUtXNEU2dzBCbDJkLXQwalo4aEVXMGtsRjgxTlg0bHRHR09wTmR0aC01aFdTZDgxaWV1dGRPTGhpQ2lJeTJBOTdHY1AzellvUXdFaWdPY0ZuWDF1SDBwWXZn with status: success
20:48:30 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Worker 9 completed https://

Starting with 172 rows...
Processing 172 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed Step 3: Downloaded 134 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 132 vs 77
    132: Techmeme - Since 2019, Brazil's courts have developed or implemented over 140 AI projects that have helped make the country's overburdened judicial system more efficient (Pedro Nakamura/Rest of World)
    77: Feedly AI - Since 2019, Brazil's courts have developed or implemented over 140 AI projects that have helped make the country's overburdened judicial system more efficient (Pedro Nakamura/Rest of World)
  Pair: 123 vs 47
    123: Reddit - Everyone's wondering if, and when, the AI bubble will pop. Here's what went down 25 years ago that ultimately burst the dot-com boom | Fortune
    47: Feedly AI - Everyone's wondering if, and when, the AI bubble will pop. Here's what went down 25 years ago that ultimately burst the dot-com boom
  Pair: 137 vs 116
    137: Techmeme - Education software company EdSights, which uses SMS texting and AI to reach students and 

In [23]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Processing 168 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1204, user_len=43
20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Using model 'gpt-4.1-mini' for summarization
20:49:13 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting summarization for 168 articles


▶ Starting Step 4: step_04_extract_summaries


20:49:19 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Extracting metadata from HTML files for 168 articles


⏱️  Total execution time: 9.81s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 168/168 articles.
💾 Summaries stored in headline DataFrame.


In [24]:
headline_df = state.headline_df
headline_df.loc[headline_df["url"]!= headline_df["final_url"]]

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags
13,Feedly AI,5 big analyst AI moves: Nvidia target upped to...,https://news.google.com/rss/articles/CBMizAFBV...,NaN,NaN,208,True,success,https://www.investing.com/news/stock-market-ne...,download/html/5_big_analyst_AI_moves__Nvidia_t...,2025-09-28T08:00:08Z,download/text/5_big_analyst_AI_moves__Nvidia_t...,10575,investing.com,Investing.com,0.0,- Barclays raised Nvidia's price target to $24...,5 big analyst AI moves: Nvidia target upped to...,[Stock Markets]
14,Feedly AI,Building smarter apps: Why machine learning se...,https://news.google.com/rss/articles/CBMiqwFBV...,NaN,NaN,124,True,success,https://azbigmedia.com/business/building-smart...,download/html/Building_smarter_apps__Why_machi...,2025-09-27T00:00:00Z,download/text/Building_smarter_apps__Why_machi...,10470,azbigmedia.com,AZ Big Media,0.0,- Machine learning is now essential across ind...,Building smarter apps: Here's why machine lear...,[]
21,Feedly AI,7 AI-Proof Jobs For Introverts Who Want Stability,https://news.google.com/rss/articles/CBMirgFBV...,NaN,NaN,138,True,success,https://www.forbes.com/sites/carolinecastrillo...,download/html/7_AI-Proof_Jobs_For_Introverts_W...,2025-09-28T20:30:00Z,download/text/7_AI-Proof_Jobs_For_Introverts_W...,9253,forbes.com,Forbes,1.0,- The article identifies seven AI-proof jobs w...,AI-proof jobs for introverts do exist. Explore...,"[Careers, artificial intelligence, AI, future ..."
29,Feedly AI,Where The AI Action Plan Falls Short On Health...,https://news.google.com/rss/articles/CBMikwFBV...,NaN,NaN,164,True,success,https://medcitynews.com/2025/09/where-the-ai-a...,download/html/Where_The_AI_Action_Plan_Falls_S...,2025-09-28T13:50:00Z,download/text/Where_The_AI_Action_Plan_Falls_S...,7570,medcitynews.com,MedCity News,0.0,- The U.S. AI Action Plan aims to build trust ...,"There are aspects of the plan worth praising, ...","[MedCity Influencers, Artificial Intelligence,..."
43,Feedly AI,"With AI Citations, Each Reference Is A Rabbit ...",https://news.google.com/rss/articles/CBMitAFBV...,NaN,NaN,176,True,success,https://www.forbes.com/sites/rheawessel/2025/0...,download/html/With_AI_Citations__Each_Referenc...,2025-09-28T16:12:53Z,download/text/With_AI_Citations__Each_Referenc...,6530,forbes.com,Forbes,1.0,- AI-generated content and citations often inc...,AI citations now mean sources of unknown origi...,[Careers]
59,Feedly AI,SuperQ Quantum Releases Post-Quantum Cryptogra...,https://news.google.com/rss/articles/CBMinwFBV...,NaN,NaN,196,True,success,https://thequantuminsider.com/2025/09/28/super...,download/html/SuperQ_Quantum_Releases_Post-Qua...,2025-09-28T10:18:00Z,download/text/SuperQ_Quantum_Releases_Post-Qua...,4898,thequantuminsider.com,The Quantum Insider,0.0,"- SuperQ Quantum released Super™ PQC Analyst, ...",SuperQ Quantum has launched Super™ PQC Analyst...,[]
61,Feedly AI,Canada Joins The Global Push For Sovereign AI ...,https://news.google.com/rss/articles/CBMivgFBV...,NaN,NaN,194,True,success,https://www.forbes.com/sites/ronschmelzer/2025...,download/html/Canada_Joins_The_Global_Push_For...,2025-09-28T14:40:10Z,download/text/Canada_Joins_The_Global_Push_For...,4823,forbes.com,Forbes,1.0,- TELUS has launched Canada's first fully sove...,TELUS launched Canada’s first fully sovereign ...,"[artificial intelligence, AI, Canada, model, s..."
66,Feedly AI,Defense tech firm Kela buys AI startup to bols...,https://news.google.com/rss/articles/CBMisgFBV...,NaN,NaN,172,True,success,https://www.timesofisrael.com/defense-tech-fir...,download/html/Defense_tech_firm_Kela_buys_AI_s...,2025-09-29T10:04:05Z,download/text/Defense_tech_firm_Kela_buys_AI_s...,4142,timesofisrael.com,Times of Israel,0.0,- Israeli defense tech company Kela Technologi...,"In its first acquisition, Kela buys Pelanor to...",[]
69,HackerNoon,AI Startup Turns Open Source Code Reviews Into...,https://ha

In [25]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


20:49:53 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
20:49:53 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Using model 'gpt-4.1-mini' for topic extraction
20:49:53 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Processing 168 articles for topic extraction


▶ Starting Step 5: step_05_cluster_by_topic


20:50:00 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Successfully extracted 557 total topics across articles
20:50:00 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== HDBSCAN Parameters ===
min_cluster_size:   6
min_samples:        3
n_components:       61
=== Clustering Quality Metrics ===
Number of clusters: 0
Noise points: 168 (100.0%)
=== Quality Scores ===

[I 2025-09-28 20:51:01,631] Trial 196 finished with value: 1.0 and parameters: {'n_components': 61, 'min_cluster_size': 6, 'min_samples': 3}. Best is trial 93 with value: -0.2073528429500118.
=== HDBSCAN Parameters ===
min_cluster_size:   5
min_samples:        3
n_components:       35
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 23 (13.7%)
Average cluster size: 72.5 ± 67.5
Cluster size range: 5 - 140
=== Quality Scores ===
Silhouette Score: 0.166 (higher is better)
Calinski-Harabasz Score: 8.0 (higher is better)
Davies-Bouldin Score: 1.038 (lower is better)
HDBSCAN Validity Index: 0.149
Composite Score: 0.157 (higher is better)

[I 2025-09-28 20:51:01,659] Trial 197 finished with value: -0.15706396272455958 and parameters: {'n_components': 35, 'min_cluster_size'

20:51:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 1: AI's Impact on Economy, Jobs, and Society
20:51:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Artificial Intelligencein Stroke Care: A Narrative Review of Diagnostic, Predictive, and Workflow Applications (Stroke Care, Diagnostic Imaging, Workflow AI, AI Validation, Bias And Fairness, Ethical AI, Policy And Regulation)
One Law Sets South Korea's AI Policy—and One Weak Link Could Break It (AI Regulation, National Strategy, Governance, Policy And Regulation, Industrial Policy, Safety And Alignment, Ethics)
Walmart CEO Issues Wake-Up Call: 'AI Is Going to Change Literally Every Job' (Job Automation, Retail Workforce, AI Impact, Chatbots, Customer Service, Jobs And Careers, Job Training)
The (economic) AI apocalypse is nigh (Economic Bubble, Investment Risks, AI Industry, Finance, Job Automation, Policy Recommendations, Hardware)
Will AI Mean Bring an End to Top Programming Language Rankings? (

20:51:02 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 

20:51:03 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | 0: Trump Shares AI-Generated MedBed Video
20:51:03 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Trump posts, then pulls bizarre AI video promoting MedBed conspiracy (Donald Trump, AI Misinformation, Deepfakes, Conspiracy Theories, Policy And Regulation, Safety And Alignment, Disinformation)
Trump Deletes Wild AI Video He Shared In Which He Promoted a Magic 'Med Bed' That Cures Diseases and Regrows Limbs (Donald Trump, Misinformation, Deepfakes, Medical Claims, Healthcare, Politics, Society And Culture)
The President posted an AI generated Fox News report announcing MedBeds, a technology that doesn’t exist (Privacy, Privacy And Surveillance, Cookies, Advertising, Policy And Regulation, Community Guidelines, Streaming)
Trump shares apparent AI video promoting ‘medbed’ conspiracy theory ()
Trump Posts and Deletes AI-Generate

⏱️  Total execution time: 70.82s
📊 Final result:
✅ Step 5 step_05_cluster_by_topic completed successfully! Organized 168 articles into topic clusters.


In [6]:
# User prompt to run workflow
# user_prompt = "Run step 6, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


21:42:40 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Rating 168 AI articles using fn_rate_articles
21:42:40 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Calculating article ratings for 168 articles
21:42:40 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Rating recency
21:42:40 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Rating spam probability
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 6: step_06_rate_articles


21:42:44 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | low quality articles: {0.0: 111, 1.0: 33, 0.9796676595602861: 2, 3.059023205018258e-07: 1, 0.020332353231731236: 1, 0.10669059229582412: 1, 0.9984987679933719: 1, 0.7057848805799208: 1, 0.9324531964618143: 1, 0.9046504317639803: 1, 2.413362771833214e-12: 1, 0.9999962302845802: 1, 0.9997694306355145: 1, 9.610239492660866e-05: 1, 0.14804720844715558: 1, 2.4300228714755076e-05: 1, 9.736200313009565e-10: 1, 2.061153622438558e-09: 1, 0.99966459043674: 1, 9.450864406622024e-13: 1, 1.3007123989067183e-05: 1, 0.43782350042481233: 1, 0.0001584362410990077: 1, 3.737889382610936e-12: 1, 0.9706877045032167: 1}
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240
21:42:48 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | on topic articles: {1.0: 107, 0.0: 6, 

21:46:09 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Generated 84 battle pairs
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | After round 5/34: 
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Number of ranking changes: 150
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Sum of absolute ranking changes: 801.0 (avg rank chg 4.77)
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | ---------------------------------------------------
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Running round 6 of 34
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | ---------------------------------------------------
21:47:07 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Generated 84 battle pairs
21:47:59 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | After round 6/34: 
21:47:59 | NewsletterAgent.test_newsl

✅ Completed Step 6: Rated 168 articles
⏱️  Total execution time: 572.86s
📊 Final result:
✅ Step 6 step_06_rate_articles completed successfully! Rated 168 articles with average rating 5.8/10.
⭐ High quality articles (≥7.0): 85
💾 Ratings stored in persistent state.


In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


21:52:16 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Starting check_workflow_status
21:52:16 | NewsletterAgent.test_newsletter_20250928201328725764 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.44s
📊 Final result:
Current workflow status:
- Progress: 66.7% (6/9 complete)
- Status summary: 6 complete, 0 started, 0 failed, 3 not started
- Next step: Step 7 — Select Sections

Step details:
- Step 1: Fetch Urls: complete
- Step 2: Filter Urls: complete
- Step 3: Download Articles: complete
- Step 4: Extract Summaries: complete
- Step 5: Cluster By Topic: complete
- Step 6: Rate Articles: complete
- Step 7: Select Sections: not_started
- Step 8: Draft Sections: not_started
- Step 9: Finalize Newsletter: not_started

Data summary:
- Total articles: 168
- AI-related: 168

Would you like me to run Step 7 (organize articles into newsletter sections) now?


In [9]:
headline_df = state.headline_df.sort_values("bradley_terry", ascending=False)
headline_df    

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,input_text,age,recency_score,low_quality,on_topic,important,bt_z,adjusted_len,rating,bradley_terry
135,VentureBeat,six different types of AI models called Liquid...,https://www.liquid.ai/press/liquid-unveils-nan...,None,None,135,True,success,https://www.liquid.ai/press/liquid-unveils-nan...,download/html/six_different_types_of_AI_models...,...,six different types of AI models called Liquid...,2.152106,-0.550032,0.000000,1.000000e+00,1.000000,2.977146,0.945321,10.0,29.191770
144,Feedly AI,"Since 2019, Brazil's courts have developed or ...",https://restofworld.org/2025/brazil-ai-courts-...,None,None,144,True,success,https://restofworld.org/2025/brazil-ai-courts-...,download/html/Since_2019__Brazil_s_courts_have...,...,"Since 2019, Brazil's courts have developed or ...",3.654640,-0.841192,0.000000,1.000000e+00,1.000000,1.936198,0.979594,10.0,19.859093
24,Feedly AI,Jensen Huang says China is 'nanoseconds behind...,https://www.tomshardware.com/jensen-huang-says...,None,None,24,True,success,https://www.tomshardware.com/jensen-huang-says...,download/html/Jensen_Huang_says_China_is__nano...,...,Jensen Huang says China is 'nanoseconds behind...,0.388876,0.527448,0.000000,1.000000e+00,1.000000,1.842454,1.008813,10.0,19.018631
43,The Verge,Google DeepMind’s new AI models can search the...,https://www.theverge.com/news/785193/google-de...,None,None,43,True,success,https://www.theverge.com/news/785193/google-de...,download/html/Google_DeepMind_s_new_AI_models_...,...,Google DeepMind’s new AI models can search the...,3.404640,-0.811144,0.000000,1.000000e+00,1.000000,1.811749,0.522575,10.0,18.743342
80,Feedly AI,Researchers (Including Google) are Betting on ...,https://slashdot.org/story/25/09/27/0632215/re...,None,None,80,True,success,https://slashdot.org/story/25/09/27/0632215/re...,download/html/Researchers__Including_Google__a...,...,Researchers (Including Google) are Betting on ...,0.000000,1.000000,0.000000,1.000000e+00,1.000000,1.754806,0.840859,10.0,18.232813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,Bloomberg,Jack Ma-Backed Fintech Giant Has an AI to Mana...,https://www.bloomberg.com/news/articles/2025-0...,None,None,21,True,success,https://www.bloomberg.com/news/articles/2025-0...,,...,Jack Ma-Backed Fintech Giant Has an AI to Mana...,1.000011,-0.000007,0.979668,1.000000e+00,0.562177,-2.273031,0.000000,0.0,-17.878992
61,WSJ,Spending on AI Is at Epic Levels. Will It Ever...,https://www.wsj.com/tech/ai/ai-bubble-building...,None,None,61,True,success,https://www.wsj.com/tech/ai/ai-bubble-building...,,...,Spending on AI Is at Epic Levels. Will It Ever...,1.000011,-0.000007,1.000000,2.018132e-01,0.000000,-2.275592,0.000000,0.0,-17.901947
78,FT,How investors should place bets on AI,https://www.ft.com/content/c610a0a0-99b2-47eb-...,None,None,78,True,success,https://www.ft.com/content/c610a0a0-99b2-47eb-...,,...,How investors should place bets on AI\n- no co...,1.000011,-0.000007,1.000000,0.000000e+00,0.000000,-2.600851,0.000000,0.0,-20.818079
134,Feedly AI,This chief people officer of a $1.5 billion AI...,https://fortune.com/2025/09/28/this-chief-peop...,None,None,134,True,success,https://fortune.com/2025/09/28/this-chief-peop...,download/html/This_chief_people_officer_of_a__...,...,This chief people officer of a $1.5 billion AI...,0.487974,0.426052,0.000000,0.000000e+00,0.000000,-2.600964,1.062206,0.0,-20.819093


In [27]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"{row.rating:.1f} [{row.title}]({row.final_url}) - {row.site_name}\n{row.summary}")) 

10.0 [six different types of AI models called Liquid Nanos](https://www.liquid.ai/press/liquid-unveils-nanos-extremely-small-foundation-models-that-match-frontier-model-quality--running-directly-on-everyday-devices) - Liquid AI
- Liquid AI has developed "Nanos," extremely small foundation models ranging from 350M to 2.6B parameters, delivering GPT-4o-class performance on specialized tasks such as multilingual translation, data extraction, mathematical reasoning, and tool-calling while running locally on everyday devices like phones and laptops.
- These task-specific models significantly reduce inference costs and energy consumption, enabling AI deployments at scale without heavy dependence on cloud infrastructure; this shift supports up to 100 AI agents per person across various devices and services, dramatically lowering the economic and environmental burdens of large AI models.
- Nanos are available via the Liquid Edge AI platform and Hugging Face under an open license, already being integrated by Fortune 500 companies across multiple industries, highlighting a future where efficient, device-level AI supports privacy, speed, and cost-effective generative applications.

10.0 [What's Next for These 3Artificial Intelligence(AI) Stocks?](https://www.fool.com/investing/2025/09/28/whats-next-for-these-3-ai-stocks/) - The Motley Fool
- Nvidia is the leading AI stock, expected to generate around $206 billion in revenue this fiscal year with robust growth driven by heavy AI infrastructure investments from major tech firms.
- Palantir Technologies shows significant net income growth due to its AI platform, but its extremely high stock valuations suggest high risk and potential overpricing.
- Apple has faced delays in AI feature rollouts but maintains strong hardware sales and market presence, giving it time to develop AI capabilities while remaining a reliable blue-chip investment.

10.0 [U.S. rejects international AI oversight at U.N. General Assembly | While heads of state and world experts highlighted risks and opportunities from AI, U.S. representatives expressed deep opposition to any global effort to govern the technology.](https://www.nbcnews.com/tech/tech-news/us-rejects-international-ai-oversight-un-general-assembly-rcna233478) - NBC News
- At the U.N. General Assembly, the U.S. opposed centralized global AI governance, advocating for national sovereignty and rejecting international regulatory control; Michael Kratsios stated bureaucratic management is not the solution.
- The U.N. launched the Global Dialogue on AI Governance to foster international cooperation and establish a global AI ecosystem, with Secretary-General António Guterres emphasizing the need to keep pace with rapidly advancing technology.
- Divergent views emerged with China and Spain supporting the U.N.'s central role in AI governance, while experts warned AI-related challenges will increasingly require transnational approaches despite the U.S. preference for bilateral and minilateral coalitions.

10.0 [AI 'Workslop' Is Plaguing American Companies, Says Stanford Research](https://www.pcmag.com/news/ai-workslop-is-plaguing-american-companies-says-stanford-research) - PCMag
- Stanford Social Media Lab and BetterUp Labs reveal over 40% of US full-time employees received "workslop," AI-generated content that appears adequate but lacks true substance, affecting productivity especially in technology and professional services.
- Workslop predominantly circulates among peers (40%) and is also sent by subordinates to managers (18%), leading about half of recipients to view colleagues who submit workslop as less creative, capable, reliable, trustworthy, and intelligent.
- Previous studies align, showing AI's modest productivity boosts are offset by additional oversight tasks, such as teachers checking for AI misuse and programmers slowed by managing AI coding tools, indicating mixed impacts of AI integration in workplaces.

10.0 [Meta has introduced revised guardrails for its AI chatbots to prevent inappropriate conversations with children](https://www.engadget.com/social-media/meta-has-introduced-revised-guardrails-for-its-ai-chatbots-to-prevent-inappropriate-conversations-with-children-200444230.html) - Engadget
- Meta has revised its AI chatbot guardrails to prevent inappropriate conversations with children, explicitly banning content that enables or endorses child sexual abuse, romantic roleplay involving minors, and advice on intimate contact with minors.
- The updated guidelines were implemented after reports revealed previous policies allowed chatbots to engage minors in romantic or sensual conversations, prompting Meta to remove such language and tighten restrictions.
- The move comes amid increased scrutiny of AI chatbots by the FTC, which has launched formal inquiries into companion AI from multiple companies to address potential harms to children and ensure safer user interactions.

...

0.0 [Gemini now explains why your Sheets formula failed](https://www.theverge.com/news/785895/gemini-explanation-google-sheets-formula) - The Verge
- Google's Gemini AI integrated into Sheets now explains formula errors and provides step-by-step instructions and alternative formula options, enhancing user productivity and understanding.
- This update aids business and personal users in managing complex data, exemplified by use cases such as tracking wedding guest RSVPs and calculating travel distances.
- While Gemini cannot directly compute some tasks like driving distances, it offers guidance using scripts or straight-line distance formulas, signaling ongoing AI capabilities expansion with practical applications in data handling.

0.0 [Wake superintendent discusses use of AI in K-12](https://nsjonline.com/article/2025/09/wake-superintendent-discusses-use-of-ai-in-k-12/) - North State Journal
- Wake County Public Schools (WCPSS) is cautiously integrating AI technologies, like Google's Gemini, for practical applications such as cataloging classroom books, improving efficiency and transparency.
- WCPSS and North Carolina are developing AI policies focused on safe, meaningful K-12 AI use, aligning with national initiatives like the Presidential AI Challenge to prepare students for AI-driven futures.
- The district emphasizes AI as a tool to enhance education, compares its integration to calculators' historical adoption, and addresses concerns about academic integrity, data privacy, and proper governance.

0.0 [7 AI-Proof Jobs For Introverts Who Want Stability](https://www.forbes.com/sites/carolinecastrillon/2025/09/28/7-ai-proof-jobs-for-introverts-who-want-stability/) - Forbes
- The article identifies seven AI-proof jobs well-suited for introverts seeking stability: software developer, data scientist, actuary, cybersecurity specialist, research scientist, occupational therapist, and therapist/counselor, all offering job growth projections from 14% to 34% through 2034 per BLS data.
- These roles require human judgment, creativity, and interpersonal nuances that AI cannot replicate, emphasizing independence, minimal client interaction, and deep focus environments, making them appealing to introverted workers.
- The future job market favors professionals who complement AI capabilities with distinctly human skills, highlighting strong career prospects and the importance of developing expertise in AI-resistant domains to ensure long-term employment security.

0.0 [ChatGPT is being ‘too nice’ to users — people want AI companions to ‘argue back occasionally’](https://nypost.com/2025/09/28/tech/chatgpt-is-being-too-nice-to-users-survey-found/) - New York Post
- A survey by Joi AI of 1,000 adults found 58% of ChatGPT users believe the AI is "too nice" and 13% find its overly polite advice nearly useless, indicating a desire for more candid and challenging interactions from AI.
- Users are seeking AI that can offer blunt honesty and pushback to mimic real human relationships, as constant agreement lacks authenticity, according to therapist Jaime Bronstein.
- Some users are developing intense emotional and romantic attachments to AI companions, with online communities like Reddit's r/MyBoyfriendIsAI documenting people experiencing complex relationships and even claiming marriage to AI personas.

0.0 [Will AI free us from life’s tedious admin?](https://www.ft.com/content/4665b369-4a46-4cf9-9c9b-bc6a6eedf0c1) - Financial Times
- The article poses the question of whether AI can alleviate tedious administrative tasks in daily life, reflecting hope that technology will serve human needs more effectively.
- It highlights various Financial Times subscription offerings, including discounted digital access plans and bundled services, targeting individual and organizational readers.
- Emphasizes the value proposition of quality journalism and expert analysis available through FT subscriptions, suggesting a market approach focused on content access and user engagement.

- rate articles
- load sources into db with reputation
- get domain from each url and put in headline_df
- look up source and reputation
- prompt for on topic , important, high quality
- run bradley terry
- combine ratings


set selected flag using rating
store to db

- next steps select sections
- clean clusters , combine clusters, select sections 